In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from tabulate import tabulate
%matplotlib inline
import os
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    recall_score,
    precision_score,
    roc_auc_score,
)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

In [2]:
geo_scores = pd.read_csv('C:/Users/Nikhil_Chamle/Desktop/P1/Fraud Detection/Geo_scores.csv')
instance = pd.read_csv('C:/Users/Nikhil_Chamle/Desktop/P1/Fraud Detection/instance_scores.csv')
lambda_wts = pd.read_csv('C:/Users/Nikhil_Chamle/Desktop/P1/Fraud Detection/Lambda_wts.csv')
qset_tats = pd.read_csv('C:/Users/Nikhil_Chamle/Desktop/P1/Fraud Detection/Qset_tats.csv')
test = pd.read_csv('C:/Users/Nikhil_Chamle/Desktop/P1/Fraud Detection/test_share.csv')
train = pd.read_csv('C:/Users/Nikhil_Chamle/Desktop/P1/Fraud Detection/train.csv')

In [3]:
print(geo_scores.head())
print("----------------------------------------------")
print(instance.head())
print("----------------------------------------------")
print(lambda_wts.head())
print("----------------------------------------------")
print(qset_tats.head())
print("----------------------------------------------")
print(test.head())
print("----------------------------------------------")
print(train.head())
print("----------------------------------------------")

       id  geo_score
0   26674       4.48
1  204314       4.48
2  176521       5.17
3   48812      -2.41
4  126870       6.55
----------------------------------------------
       id  instance_scores
0  173444            -0.88
1  259378             1.50
2  161170             0.44
3  191161             0.76
4   34521            -0.84
----------------------------------------------
     Group  lambda_wt
0   Grp936       3.41
1   Grp347      -2.88
2   Grp188       0.39
3  Grp1053      -2.75
4    Grp56      -0.83
----------------------------------------------
       id  qsets_normalized_tat
0    9983                  2.41
1  266000                  3.10
2   77525                  1.03
3  160765                -11.63
4  138220                 -4.48
----------------------------------------------
       id   Group      Per1      Per2      Per3      Per4      Per5      Per6  \
0  146574  Grp229 -0.300000  1.540000  0.220000 -0.280000  0.570000  0.260000   
1  268759  Grp141  0.633333  0.953333 

In [4]:
print(geo_scores.shape)
print("----------------------------------------------")
print(instance.shape)
print("----------------------------------------------")
print(lambda_wts.shape)
print("----------------------------------------------")
print(qset_tats.shape)
print("----------------------------------------------")
print(test.shape)
print("----------------------------------------------")
print(train.shape)
print("----------------------------------------------")

(1424035, 2)
----------------------------------------------
(1424035, 2)
----------------------------------------------
(1400, 2)
----------------------------------------------
(1424035, 2)
----------------------------------------------
(56962, 27)
----------------------------------------------
(227845, 28)
----------------------------------------------


In [5]:
print(geo_scores.isnull().sum())
print("----------------------------------------------")
print(instance.isnull().sum())
print("----------------------------------------------")
print(lambda_wts.isnull().sum())
print("----------------------------------------------")
print(qset_tats.isnull().sum())
print("----------------------------------------------")
print(test.isnull().sum())
print("----------------------------------------------")
print(train.isnull().sum())
print("----------------------------------------------")

id               0
geo_score    71543
dtype: int64
----------------------------------------------
id                 0
instance_scores    0
dtype: int64
----------------------------------------------
Group        0
lambda_wt    0
dtype: int64
----------------------------------------------
id                           0
qsets_normalized_tat    103201
dtype: int64
----------------------------------------------
id                0
Group             0
Per1              0
Per2              0
Per3              0
Per4              0
Per5              0
Per6              0
Per7              0
Per8              0
Per9              0
Dem1              0
Dem2              0
Dem3              0
Dem4              0
Dem5              0
Dem6              0
Dem7              0
Dem8              0
Dem9              0
Cred1             0
Cred2             0
Cred3             0
Cred4             0
Cred5             0
Cred6             0
Normalised_FNT    0
dtype: int64
-----------------------------------

In [6]:
print(geo_scores.isnull().sum()/len(geo_scores)*100)
print("----------------------------------------------")
print(qset_tats.isnull().sum()/len(qset_tats)*100)

id           0.000000
geo_score    5.023964
dtype: float64
----------------------------------------------
id                      0.000000
qsets_normalized_tat    7.247083
dtype: float64


In [7]:
print(geo_scores.info())
print("----------------------------------------------")
print(qset_tats.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1424035 entries, 0 to 1424034
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   id         1424035 non-null  int64  
 1   geo_score  1352492 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 21.7 MB
None
----------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1424035 entries, 0 to 1424034
Data columns (total 2 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1424035 non-null  int64  
 1   qsets_normalized_tat  1320834 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 21.7 MB
None


In [8]:
print(geo_scores.describe())
print("----------------------------------------------")
print(qset_tats.describe())

                 id     geo_score
count  1.424035e+06  1.352492e+06
mean   1.424030e+05 -9.279168e-06
std    8.221673e+04  7.827199e+00
min    0.000000e+00 -1.093900e+02
25%    7.120100e+04 -5.860000e+00
50%    1.424030e+05  1.800000e-01
75%    2.136050e+05  5.860000e+00
max    2.848060e+05  4.581000e+01
----------------------------------------------
                 id  qsets_normalized_tat
count  1.424035e+06          1.320834e+06
mean   1.424030e+05          1.094006e-05
std    8.221673e+04          7.731794e+00
min    0.000000e+00         -1.404400e+02
25%    7.120100e+04         -5.860000e+00
50%    1.424030e+05          2.000000e-02
75%    2.136050e+05          5.860000e+00
max    2.848060e+05          6.110000e+01


In [9]:
print(geo_scores.mean())
print("----------------------------------------------")
print(qset_tats.mean())

id           142403.000000
geo_score        -0.000009
dtype: float64
----------------------------------------------
id                      142403.000000
qsets_normalized_tat         0.000011
dtype: float64


In [10]:
print(geo_scores.median())
print("----------------------------------------------")
print(qset_tats.median())

id           142403.00
geo_score         0.18
dtype: float64
----------------------------------------------
id                      142403.00
qsets_normalized_tat         0.02
dtype: float64


In [11]:
geo_scores.fillna(0.18, inplace=True)
qset_tats.fillna(0.02, inplace=True)

In [12]:
train['data'] = 'train'
test['data'] = 'test'

In [13]:
test.head()

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Dem8,Dem9,Cred1,Cred2,Cred3,Cred4,Cred5,Cred6,Normalised_FNT,data
0,146574,Grp229,-0.300000,1.540000,0.220000,-0.280000,0.570000,0.260000,0.700000,1.076667,...,0.546667,0.313333,0.703333,0.813333,0.776667,0.796667,0.823333,0.783333,-249.7500,test
1,268759,Grp141,0.633333,0.953333,0.810000,0.466667,0.910000,0.253333,1.040000,0.550000,...,0.770000,0.993333,0.536667,0.703333,0.806667,0.630000,0.673333,0.673333,-249.8125,test
2,59727,Grp188,1.043333,0.740000,0.860000,1.006667,0.583333,0.616667,0.630000,0.686667,...,0.756667,0.953333,0.623333,0.753333,0.870000,0.596667,0.680000,0.670000,-248.1200,test
3,151544,Grp426,1.283333,0.300000,0.576667,0.636667,0.256667,0.543333,0.356667,0.663333,...,0.460000,0.260000,0.800000,0.606667,0.456667,0.320000,0.676667,0.660000,-222.9875,test
4,155008,Grp443,1.186667,0.326667,0.476667,0.866667,0.436667,0.680000,0.476667,0.686667,...,0.740000,0.823333,0.670000,0.896667,0.566667,0.546667,0.650000,0.663333,-196.2200,test


In [14]:
train.shape

(227845, 29)

In [15]:
test.shape

(56962, 28)

In [16]:
# append both train and test data together

all_data = pd.concat([train, test], axis=0)

In [17]:
all_data.head()

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Dem9,Cred1,Cred2,Cred3,Cred4,Cred5,Cred6,Normalised_FNT,Target,data
0,112751,Grp169,1.070000,0.580000,0.480000,0.766667,1.233333,1.993333,0.340000,1.010000,...,0.726667,0.606667,1.010000,0.933333,0.603333,0.686667,0.673333,-245.7500,0.0,train
1,18495,Grp161,0.473333,1.206667,0.883333,1.430000,0.726667,0.626667,0.810000,0.783333,...,0.743333,0.680000,0.690000,0.560000,0.670000,0.553333,0.653333,-248.0000,0.0,train
2,23915,Grp261,1.130000,0.143333,0.946667,0.123333,0.080000,0.836667,0.056667,0.756667,...,0.820000,0.600000,0.383333,0.763333,0.670000,0.686667,0.673333,-233.1250,0.0,train
3,50806,Grp198,0.636667,1.090000,0.750000,0.940000,0.743333,0.346667,0.956667,0.633333,...,0.900000,0.680000,0.846667,0.423333,0.520000,0.846667,0.760000,-249.7775,0.0,train
4,184244,Grp228,0.560000,1.013333,0.593333,0.416667,0.773333,0.460000,0.853333,0.796667,...,0.486667,0.693333,0.526667,0.520000,0.716667,0.706667,0.673333,-247.5775,0.0,train


In [18]:
all_data.isnull().sum()

id                    0
Group                 0
Per1                  0
Per2                  0
Per3                  0
Per4                  0
Per5                  0
Per6                  0
Per7                  0
Per8                  0
Per9                  0
Dem1                  0
Dem2                  0
Dem3                  0
Dem4                  0
Dem5                  0
Dem6                  0
Dem7                  0
Dem8                  0
Dem9                  0
Cred1                 0
Cred2                 0
Cred3                 0
Cred4                 0
Cred5                 0
Cred6                 0
Normalised_FNT        0
Target            56962
data                  0
dtype: int64

In [19]:
all_data['id'].nunique()

284807

In [20]:
geo_scores = geo_scores.groupby('id').mean()

In [21]:
instance = instance.groupby('id').mean()

In [22]:
qset_tats = qset_tats.groupby('id').mean()

In [23]:
# append geo_scores , instance and qset_tats with all_dataset

all_data = pd.merge(all_data, geo_scores , on='id', how='left')

In [24]:
all_data = pd.merge(all_data, instance , on='id', how='left')

In [25]:
all_data = pd.merge(all_data, qset_tats , on='id', how='left')

In [26]:
all_data.tail()

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Cred3,Cred4,Cred5,Cred6,Normalised_FNT,Target,data,geo_score,instance_scores,qsets_normalized_tat
284802,18333,Grp102,0.553333,1.043333,1.096667,0.686667,0.673333,0.340000,0.900000,0.643333,...,0.610000,0.690000,0.750000,0.700000,-249.5050,NaN,test,-0.114,-0.51,0.010
284803,244207,Grp504,1.353333,0.616667,0.276667,0.783333,0.690000,0.650000,0.473333,0.670000,...,0.640000,0.883333,0.663333,0.660000,-248.7525,NaN,test,0.110,-0.46,0.064
284804,103277,Grp78,1.083333,0.433333,0.806667,0.490000,0.243333,0.316667,0.533333,0.606667,...,0.603333,0.880000,0.643333,0.676667,-231.0500,NaN,test,-0.530,0.65,-0.020
284805,273294,Grp134,0.566667,1.153333,0.370000,0.616667,0.793333,0.226667,0.910000,0.696667,...,0.566667,0.616667,0.713333,0.706667,-246.3150,NaN,test,0.270,-0.92,0.610
284806,223337,Grp18,1.426667,0.110000,-0.006667,-0.200000,0.983333,1.870000,0.033333,0.963333,...,0.586667,0.616667,0.683333,0.650000,-248.4500,NaN,test,-0.664,1.57,0.354


In [27]:
all_data['Group'].nunique()

1400

In [28]:
all_data = pd.merge(all_data, lambda_wts , on='Group', how='left')
all_data

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Cred4,Cred5,Cred6,Normalised_FNT,Target,data,geo_score,instance_scores,qsets_normalized_tat,lambda_wt
0,112751,Grp169,1.070000,0.580000,0.480000,0.766667,1.233333,1.993333,0.340000,1.010000,...,0.603333,0.686667,0.673333,-245.7500,0.0,train,0.256,-0.06,-0.700,-0.13
1,18495,Grp161,0.473333,1.206667,0.883333,1.430000,0.726667,0.626667,0.810000,0.783333,...,0.670000,0.553333,0.653333,-248.0000,0.0,train,-0.250,0.52,0.140,0.66
2,23915,Grp261,1.130000,0.143333,0.946667,0.123333,0.080000,0.836667,0.056667,0.756667,...,0.670000,0.686667,0.673333,-233.1250,0.0,train,-0.950,1.56,-0.430,-0.51
3,50806,Grp198,0.636667,1.090000,0.750000,0.940000,0.743333,0.346667,0.956667,0.633333,...,0.520000,0.846667,0.760000,-249.7775,0.0,train,0.490,0.70,-0.302,0.72
4,184244,Grp228,0.560000,1.013333,0.593333,0.416667,0.773333,0.460000,0.853333,0.796667,...,0.716667,0.706667,0.673333,-247.5775,0.0,train,0.850,-0.47,-0.630,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,18333,Grp102,0.553333,1.043333,1.096667,0.686667,0.673333,0.340000,0.900000,0.643333,...,0.690000,0.750000,0.700000,-249.5050,NaN,test,-0.114,-0.51,0.010,-0.40
284803,244207,Grp504,1.353333,0.616667,0.276667,0.783333,0.690000,0.650000,0.473333,0.670000,...,0.883333,0.663333,0.660000,-248.7525,NaN,test,0.110,-0.46,0.064,-1.54
284804,103277,Grp78,1.083333,0.433333,0.806667,0.490000,0.243333,0.316667,0.533333,0.606667,...,0.880000,0.643333,0.676667,-231.0500,NaN,test,-0.530,0.65,-0.020,0.29
284805,273294,Grp134,0.566667,1.153333,0.370000,0.616667,0.793333,0.226667,0.910000,0.696667,...,0.616667,0.713333,0.706667,-246.3150,NaN,test,0.270,-0.92,0.610,-0.69


In [29]:
# Split train and test data seperately

train = all_data[all_data['data']=='train']
test = all_data[all_data['data']=='test']

In [30]:
print(train.shape)
print()
print(test.shape)

(227845, 33)

(56962, 33)


In [31]:
test = test.drop(['id', 'Group','Target', 'data'], axis=1)

In [32]:
# split the data into independent and dependent variable 

x = train.drop(['id', 'Group','Target', 'data'], axis=1)
y = train[['Target']]

In [33]:
smote = SMOTE(random_state=101, sampling_strategy=0.75)
x_smote, y_smote = smote.fit_resample(x, y)

In [34]:
print(x_smote.shape, y_smote.shape)
print()
print(y.value_counts())
print()
print(y_smote.value_counts())

(398039, 29) (398039, 1)

Target
0.0       227451
1.0          394
dtype: int64

Target
0.0       227451
1.0       170588
dtype: int64


In [35]:
# split the data into train and test for building the model and prediction

X_train, X_test, y_train, y_test = train_test_split(x_smote, y_smote, train_size=0.7, 
                                                    random_state=101, stratify=y_smote)

In [36]:
#model training and prediction

models = []
models.append(("LR", LogisticRegression()))
models.append(("LDA", LinearDiscriminantAnalysis()))
models.append(("KNN", KNeighborsClassifier()))
models.append(("NB", GaussianNB()))
models.append(("RF", RandomForestClassifier()))

results = []
names = []

for name, model in models:
    kfold = KFold(n_splits=10)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring="accuracy")
    results.append(cv_results)
    names.append(name)
    msg = "%s Accuracy: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

    # Calculating training accuracy
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    print(f"Training Accuracy for {name}: {train_accuracy}")
    print()

    # Calculating test accuracy
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    print(f"Test Accuracy for {name}: {test_accuracy}")

    # Calculating additional metrics
    print(f"Confusion Matrix for {name}:\n{confusion_matrix(y_test, y_pred)}")
    print()
    print(f"Classification Report for {name}:\n{classification_report(y_test, y_pred)}")
    print()
    print(f"Recall for {name}: {recall_score(y_test, y_pred)}")
    print()
    print(f"Precision for {name}: {precision_score(y_test, y_pred)}")
    print()
    print(f"ROC AUC for {name}: {roc_auc_score(y_test, y_pred)}")
    print("----------------------------------------------------------------------------")

LR Accuracy: 0.952693 (0.001337)
Training Accuracy for LR: 0.9534682568451729

Test Accuracy for LR: 0.9544266907848458
Confusion Matrix for LR:
[[67157  1078]
 [ 4364 46813]]

Classification Report for LR:
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     68235
         1.0       0.98      0.91      0.95     51177

    accuracy                           0.95    119412
   macro avg       0.96      0.95      0.95    119412
weighted avg       0.96      0.95      0.95    119412


Recall for LR: 0.9147273189127929

Precision for LR: 0.9774905514606085

ROC AUC for LR: 0.949464487477207
----------------------------------------------------------------------------
LDA Accuracy: 0.924803 (0.001603)
Training Accuracy for LDA: 0.9248672957035751

Test Accuracy for LDA: 0.9260710816333367
Confusion Matrix for LDA:
[[67668   567]
 [ 8261 42916]]

Classification Report for LDA:
              precision    recall  f1-score   support

         0.0  